In [17]:
## With Hadoop hdfs
# remote (on docker) mlflow-server
# remote(on docker) postgresql for backend-store
# remote(on docker) hdfs for artifact-store. But Hadoop will be installed on locally because pyarrow needs to some hadoop file.

In [18]:
import os

os.environ['HADOOP_HOME'] = '/home/hdoop/hadoop'
os.environ['ARROW_LIBHDFS_DIR'] = '/home/hdoop/hadoop/lib/native'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

import mlflow

mlflow.set_tracking_uri("http://localhost:5000")

print(mlflow.list_experiments())

[<Experiment: artifact_location='hdfs://localhost:9000/Models/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("Remote_Postgresql-Local_Hdfs-Local-Mlflow")

with mlflow.start_run():
    print(mlflow.get_artifact_uri())

    x, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_param('Params',params)

    lr = LogisticRegression(**params)
    lr = lr.fit(x, y)	
	
    y_pred = lr.predict(x)
    
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
	
    mlflow.sklearn.log_model(lr, artifact_path='')
    print(f'default artifact URI: {mlflow.get_artifact_uri()}')

2022/11/05 18:29:29 INFO mlflow.tracking.fluent: Experiment with name 'Remote_Postgresql-Local_Hdfs-Local-Mlflow' does not exist. Creating a new experiment.


hdfs://localhost:9000/Models/1/c836fa1e7eba46e1bc9a0353c28530f5/artifacts


/home/mmucahitnas/anaconda3/envs/MLOps/lib/python3.8/site-packages/mlflow/store/artifact/hdfs_artifact_repo.py:182: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  connected = pa.hdfs.connect(
2022-11-05 18:29:31,217 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2022-11-05 18:29:31,274 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2022-11-05 18:29:31,288 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2022-11-05 18:29:31,310 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2022-11-05 18:29:31,324 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


default artifact URI: hdfs://localhost:9000/Models/1/c836fa1e7eba46e1bc9a0353c28530f5/artifacts


In [ ]:
import pyarrow

os.environ['HADOOP_HOME'] = '/home/mmucahitnas/Downloads/hadoop-3.3.3/'
os.environ['ARROW_LIBHDFS_DIR'] = '/home/mmucahitnas/Downloads/lib/native'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

hdfs = pyarrow.hdfs.connect(host='172.17.0.2', port=9000)

In [ ]:
hdfs.ls('/')

In [ ]:
## AWS

# Remote ec2 server for mlflow-server
# Remote Postgresql for backends-store
# Remote S3 for artifacts-store

In [ ]:
import mlflow

TRACKING_SERVER_HOST = "ec2-44-203-192-141.compute-1.amazonaws.com"
mlflow.set_tracking_uri(f'http://{TRACKING_SERVER_HOST}:5000')

f'tracking URI: {mlflow.get_tracking_uri()}'

In [ ]:
mlflow.list_experiments()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("S3-Postgresql-Remote-AWS")

with mlflow.start_run():
    print(mlflow.get_artifact_uri())

    x, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    # mlflow.log_param(params)

    lr = LogisticRegression(**params)
    lr = lr.fit(x, y)	
	
    y_pred = lr.predict(x)
    
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
	
    mlflow.sklearn.log_model(lr, artifact_path='Models')
    print(f'default artifact URI: {mlflow.get_artifact_uri()}')

In [ ]:
mlflow.list_experiments()